In [1]:
!pip install scikit-learn-extra

In [2]:
!pip install -q pyyaml h5py

In [3]:
#import neccesary packages
import tensorflow_hub as hub
import tensorflow as tf
import pickle
from keras import backend as K
import numpy as np
from sklearn_extra.cluster import KMedoids
from tensorflow import keras
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from datetime import datetime
from scipy.spatial import distance_matrix
import sys

In [4]:
#mount your google drive here
print(tf.__version__)
from google.colab import drive 
drive.mount('/content/drive')

2.6.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import datasets

In [5]:
#directory of the datasets. Github provides these files in the form of pickle files
#dir = "../datasets/hotel/"
dir = "./"
with open (dir + 'y_train', 'rb') as fp:
    y_train = pickle.load(fp)

with open (dir + 'train_not_clean', 'rb') as fp:
    train_not_clean = pickle.load(fp)

with open (dir + 'test_not_clean', 'rb') as fp:
    test_not_clean = pickle.load(fp)

with open (dir + 'y_test', 'rb') as fp:
    y_test = pickle.load(fp)

In [6]:
print(train_not_clean[:10])
print(y_train[:10])

["RENT A STEAM CLEANER. The location is very, very inconvenient, but I am sure that is nothing they can help, it is just how the city has grown around their location. I would have rated the room as a 4 or 5 for clean if the sofa was not so disgusting. I literally covered it with a sheet just to sit on it. There was not one inch of surface that was not covered with a greasy/dirty stain. I would have switched rooms if I hadn't already unloaded all of my stuff in the rain before I turned on the lights to reveal the nasty sofa. Otherwise, the room was clean and comfortable. It would be worth a couple of hundred dollars to get a steam cleaner and clean the sofas in each room. Not to mention how unsanitary it is", "Great hotel and staff - recommend. I came in late after a long drive from Vegas ending in a whole lot of highway exits and concentration to get to Napa. I was greeted by a friendly night staff who was expecting me, and (not sure of his name) made me feel more relaxed immediately. 

# Data preprocessing

In [7]:
#this method is to split the paragraphs into sentences
def gen_sents(para):
    res = []
    for p in para:
        sents = p.split(".")
        res.append(sents)
    return res


train_noclean_sents = gen_sents(train_not_clean)
test_noclean_sents = gen_sents(test_not_clean)

In [8]:
x_train = train_noclean_sents
x_test = test_noclean_sents

#optional: just to make sure the label values are integers
y_train = [int(y) for y in y_train]
y_test = [int(y) for y in y_test]


print('test_noclean_sents ', len(test_noclean_sents), len(test_noclean_sents[0]), test_noclean_sents[0])
print('y_train: ', y_train[:10])
print('y_test: ', y_test[:10])

test_noclean_sents  902 7 ['Share your room with roach or bug', ' Roach in the room but the staff said its just a bug not a roach, like that made it any better', ' Big hole in the bathroom wall were this bug/roach came out of', ' Wonder what other rodent will come out of the wall', 'Thank you for providing us with your feedback', ' I would like to apologize for any of the inconveniences you may have experienced during your time with us', ' I wish that I had been notified of your concerns before you had checked out and I would like to apologize on behalf of our staff']
y_train:  [1, 1, 1, 0, 1, 1, 0, 0, 1, 1]
y_test:  [0, 0, 1, 1, 0, 1, 0, 0, 0, 1]


In [9]:
#import Google Sentence encoder, to convert sentences into vector values
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)

def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [10]:
#sampling training data (sentences)
sample_sentences = []
#choose with data to sample
for p in train_noclean_sents[:1000]:
    sample_sentences.extend(p)

#compute vector values of sentences
sample_sent_vect = embed(sample_sentences)
print(sample_sent_vect)

tf.Tensor(
[[-0.02057868 -0.02088437  0.04719457 ... -0.05737035  0.0809434
   0.03041554]
 [ 0.00906795 -0.03493962 -0.02079896 ... -0.04224585  0.03169224
   0.0206266 ]
 [-0.05278953  0.00524478 -0.04006644 ... -0.05303752  0.07519065
  -0.00645197]
 ...
 [-0.01198283 -0.03123995 -0.04763543 ... -0.02125541 -0.00142226
   0.0453642 ]
 [ 0.001819   -0.06974981 -0.0585594  ...  0.03950792  0.06799415
   0.08365905]
 [-0.0730544  -0.02952773  0.03002902 ... -0.05277651  0.01056046
   0.04155547]], shape=(11747, 512), dtype=float32)


# Prototype initialization

In [11]:
k_protos, vect_size = 10, 512
kmedoids = KMedoids(n_clusters=k_protos, random_state=0).fit(sample_sent_vect)
k_cents = kmedoids.cluster_centers_
print(k_cents.shape)

(10, 512)


# Model training and testing

In [12]:
#import protoryNet, we recommend you download the protoryNet.py file from Github to have the most updated version
from protoryNet import ProtoryNet

In [13]:
pNet = ProtoryNet() 

In [14]:
model = pNet.createModel(k_cents)

[db] model.input =  KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name='input_1'), name='input_1', description="created by layer 'input_1'")
[db] protoLayerName =  proto_layer
[db] protoLayer =  <protoryNet.ProtoryNet.createModel.<locals>.prototypeLayer object at 0x7f19f9dc2610>
[db] protoLayer.output =  (<KerasTensor: shape=(None, None, 10) dtype=float32 (created by layer 'proto_layer')>, <KerasTensor: shape=(10, 512) dtype=float32 (created by layer 'proto_layer')>)
[db] distanceLayer.output =  KerasTensor(type_spec=TensorSpec(shape=(None, None, 10), dtype=tf.float32, name=None), name='distance_layer/PartitionedCall:0', description="created by layer 'distance_layer'")
Model: "custom_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer (Kera

In [15]:
pNet.train(x_train,y_train,x_test,y_test)

Epoch  0
i =   0
1/1 [==============================] - 14s 14s/step - loss: 0.7004
Evaluating y_pred, y  [0.4889834] 0 0
Evaluating y_pred, y  [0.48898146] 0 1
Evaluate on valid set:  0.42017738359201773
This is the best eval res, saving the model...
saving model now = 2021-08-23 11:48:34.335129
date and time = 23_08_2021_11_48_34
just saved
i =   50
1/1 [==============================] - 0s 218ms/step - loss: 0.6529
i =   100
1/1 [==============================] - 0s 221ms/step - loss: 0.7873
i =   150
1/1 [==============================] - 0s 222ms/step - loss: 0.6065
i =   200
1/1 [==============================] - 0s 216ms/step - loss: 0.8340
Evaluating y_pred, y  [0.5535297] 1 0
Evaluating y_pred, y  [0.6039297] 1 1
Evaluate on valid set:  0.5210643015521065
This is the best eval res, saving the model...
saving model now = 2021-08-23 11:51:33.419058
date and time = 23_08_2021_11_51_33
just saved
i =   250
1/1 [==============================] - 0s 222ms/step - loss: 1.0013
i =   3

KeyboardInterrupt: ignored

In [16]:
new_protos = pNet.projection(sample_sent_vect)

[db] p_count =  0
[db] p_count =  1
[db] p_count =  2
[db] p_count =  3
[db] p_count =  4
[db] p_count =  5
[db] p_count =  6
[db] p_count =  7
[db] p_count =  8
[db] p_count =  9


In [17]:
print(len(new_protos))

10


In [18]:
pNet.prototypeInterpretation(sample_sentences,sample_sent_vect)

[db] p_count =  0
[db] p_count =  1
[db] p_count =  2
[db] p_count =  3
[db] p_count =  4
[db] p_count =  5
[db] p_count =  6
[db] p_count =  7
[db] p_count =  8
[db] p_count =  9
p_count =  0
count =  9935
p_count =  1
count =  9935
p_count =  2
count =  9935
p_count =  3
count =  9935
p_count =  4
count =  9935
p_count =  5
count =  9935
p_count =  6
count =  9935
p_count =  7
count =  9935
p_count =  8
count =  9935
p_count =  9
count =  9935
prototype index =  0
(453, 0.0)  Still, I thought the hotel was an excellent value for the money, and I would definitely stay there again
(2972, 0.7802615)  Overall I was very happy with the hotel and would definitely stay here again
(6958, 0.805239)  All in all, i'd recommend this hotel and definitely stay there again
(5074, 0.83005947) It is certainly not cheap, but I liked the hotel and I would stay there again the next time I go back to Philadelphia
(2126, 0.8513434)  Overall a great stay here and even though it didn't have a balcony I'd de

In [ ]:
#pNet.protoFreq(sample_sent_vect)

In [ ]:
#pNet.pruningTrain(2,x_train,y_train,x_test,y_test)